In [10]:
import cv2
import os
from random import *
from numpy import *
import numpy as np
import math
import glob
from numba import jit, prange
import time
import warnings
from numba import cuda
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from numba.typed import List
from argparse import ArgumentParser
parser = ArgumentParser()

@jit(nopython=True)
def Random_Centroids(k, image, height, width):
    Centroids_R,Centroids_G, Centroids_B = np.array([1] * k), np.array([1] * k), np.array([1] * k)
    
    for i in range(k):
        Centroids_R[i] = image[randrange(width)][randrange(height)][0]
        Centroids_G[i] = image[randrange(width)][randrange(height)][1]
        Centroids_B[i] = image[randrange(width)][randrange(height)][2]
    
    return Centroids_R, Centroids_G, Centroids_B

@cuda.jit
def Choose_Centroid(Index_Map, Map_Centroids, image, Centroids):
  
    row, col = cuda.grid(2)
    
    if row < image.shape[0] and col < image.shape[1]:
        pixel = image[row][col]
        index = 0
        temp = 999

        for i in range(len(Centroids)):
            distance = (math.sqrt((math.pow(int(Centroids[i][0])-int(pixel[0]),2))+(math.pow(int(Centroids[i][1])-int(pixel[1]),2))+(math.pow(int(Centroids[i][2])-int(pixel[2]),2))))
            if (distance <= temp):
                temp = distance
                index = i
        
        Index_Map[row][col][0] = index
        Map_Centroids[row][col][0] = Centroids[index][0]
        Map_Centroids[row][col][1] = Centroids[index][1]
        Map_Centroids[row][col][2] = Centroids[index][2]
        
@cuda.jit
def ReChoose_Centroid(index_map, image, k, Centroids_):
    pos = cuda.grid(1)
    
    for row in range(image.shape[0]):
        for col in range(image.shape[1]):
            temp = index_map[row][col]
            
            if (pos < 3):
                Centroids_[pos][temp[0]] += image[row][col][pos]
            if (pos < 6 and pos >= 3):
                Centroids_[pos][temp[0]] += 1   
            
def SegmentationImage(img, k):
    
    timing_choose = 0
    timing_rechoose = 0
    #img = cv2.imread(image)
    height, width, channels = img.shape

    Centroids_R, Centroids_G, Centroids_B = Random_Centroids(k, img, width, height)

    #First Centroids
    Centroids = np.array([[1, 2, 3]] * k, np.int64)
    
    for i in range(k):
        Centroids[i] = [Centroids_R[i], Centroids_G[i], Centroids_B[i]]

    #KMeans
    for i in range(10):
        
        image_global_mem = cuda.to_device(img)
        Centroids_global_mem = cuda.to_device(Centroids)

        Map_Centroids_global_mem = cuda.device_array((height, width, 3))
        Index_Map_global_mem = cuda.device_array((height, width, 1))

        threadsperblock = (32,32)
        blockspergrid_x = int(math.ceil(height / threadsperblock[0]))
        blockspergrid_y = int(math.ceil(width / threadsperblock[1]))
        blockspergrid = (blockspergrid_x, blockspergrid_y)
        
        start_time1 = time.time()
        Choose_Centroid[blockspergrid, threadsperblock](Index_Map_global_mem, Map_Centroids_global_mem, image_global_mem, Centroids_global_mem)
        timing_choose = timing_choose + (time.time() - start_time1)
        
        Index_Map = Index_Map_global_mem.copy_to_host().astype(int) 
    
        start_time2 = time.time()
        Index_Map_global_mem = cuda.to_device(Index_Map)
        
        Centroids_R,Centroids_G, Centroids_B = np.array([0] * k), np.array([0] * k), np.array([0] * k)
        n_Centroids_R, n_Centroids_G, n_Centroids_B = np.array([0] * k), np.array([0] * k), np.array([0] * k)
        
        Centroids_ = [Centroids_R,Centroids_G, Centroids_B, n_Centroids_R, n_Centroids_G, n_Centroids_B]
        Centroids__global_mem = cuda.to_device(Centroids_)
         
        ReChoose_Centroid[1, 6](Index_Map_global_mem, image_global_mem, k, Centroids__global_mem)
        Centroids_ = Centroids__global_mem.copy_to_host()    
        timing_rechoose = timing_rechoose + (time.time() - start_time2)
           
        Centroids_ = Centroids_.astype(np.float).astype("Int32")
        
        for i in range(k):
            for j in range(3,6):
                if(Centroids_[j][i] == 0):
                    Centroids_[j][i] = 1
          
            Centroids[i] = [Centroids_[0][i]/Centroids_[3][i], Centroids_[1][i]/Centroids_[4][i], Centroids_[2][i]/Centroids_[5][i]]
        
        Map_Centroids = Map_Centroids_global_mem.copy_to_host().astype(int)
        
#     print("--- Choose Time: %s seconds ---" % timing_choose)
#     print("--- ReChoose Time: %s seconds ---" % timing_rechoose)
    return Map_Centroids
    

In [44]:
def Break_Video_To_Frames(video_path):
  count = 0
  vidcap = cv2.VideoCapture(video_path)
  success,image = vidcap.read()
  success = True

  while success:
    vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*100))    # added this line 
    success,image = vidcap.read()
    try:
      cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file
      count = count + 1
    except:
      break

In [20]:
def Resize_Image(orginal_image, scale_percent):
  #print('Original Dimensions : ', orginal_image.shape)
 
  width = int(orginal_image.shape[1] * scale_percent / 100)
  height = int(orginal_image.shape[0] * scale_percent / 100)
  dim = (width, height)
  
  # resize image
  resized_image = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
 
  #print('Resized Dimensions : ', resized_image.shape)
  return resized_image

In [21]:
def Calc_K(SSE):
  temp = []
  for i in range(len(SSE) - 3):
    temp.append(abs(int(SSE[i] - SSE[i+1]) - int(SSE[i+1] - SSE[i+2]) - int(SSE[i+2] - SSE[i+3])))
  print(temp)
  print(min(temp))
  k = temp.index(min(temp)) + 1 
  return k

In [30]:
def Calc_SSE_Image(index_image, n, img, scale_percent):
  start_time = time.time()
  SSE = [0] * n

  for i in range(n):
    SSE[i] = 0

    resized_image = Resize_Image(img, scale_percent)
    out_img = SegmentationImage(resized_image, i + 1)

    for x in range(len(resized_image)):
      for y in range(len(resized_image[0])):
        SSE[i] = SSE[i] + math.sqrt(math.pow((int(resized_image[x][y][0])-int(out_img[x][y][0])),2) + math.pow((int(resized_image[x][y][1])-int(out_img[x][y][1])),2) + math.pow((int(resized_image[x][y][2])-int(out_img[x][y][2])),2))
     
  k = Calc_K(SSE)
  print('--- Calculating SSE image = {}, k = {} with time = {} ----'.format(index_image, k, time.time() - start_time)) 
  return k

In [54]:
def Segmentation_Video(scale_percent):
  start_time = time.time()
  n = 10
  SSE = [0] * n
  images = glob.glob("*.jpg")
  count = 0
  for i in images:
    if("frame" in i):
      print("Coverting {}/{}".format(count, len(images)))
      img = cv2.imread(i)
      k = Calc_SSE_Image(count, n, img, scale_percent)
      out_img = SegmentationImage(img, k)
      cv2.imwrite(i.replace('frame','output'), out_img)
      count = count + 1

  print("--- Done in %s seconds ---" % (time.time() - start_time))

In [52]:
def Merge_Frames_To_Video(output_filename):
  img_array = []
  height = 0
  width = 0
  images = glob.glob("*.jpg")
  for filename in images:
    if ('output' in filename):
      img = cv2.imread(filename)
      height, width, layers = img.shape
      img_array.append(img)

  fourcc = cv2.VideoWriter_fourcc(*'MPEG')
  out = cv2.VideoWriter('output.avi', fourcc, 10, (width,height))
  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release() 

In [53]:
import os     

def Clean_Images():
  for i in glob.glob("*.jpg"):
    os.remove(i)

Clean_Images()

In [ ]:
Break_Video_To_Frames("Video.mp4")

Segmentation_Video(scale_percent = 20)

Merge_Frames_To_Video('output.avi')

Clean_Images()

Coverting 0/101
[1053786, 537053, 131652, 800358, 367824, 0, 29830]
0
--- Calculating SSE image = 0, k = 6 with time = 2.062856435775757 ----
Coverting 1/101
[1053786, 330050, 100890, 311931, 232542, 64652, 250407]
64652
--- Calculating SSE image = 1, k = 6 with time = 1.693131446838379 ----
Coverting 2/101
[1185438, 119430, 30762, 166540, 246963, 68139, 1656]
1656
--- Calculating SSE image = 2, k = 7 with time = 1.6369590759277344 ----
Coverting 3/101
[1053786, 139676, 124880, 5605, 10587, 45031, 24412]
5605
--- Calculating SSE image = 3, k = 4 with time = 1.6150047779083252 ----
Coverting 4/101
[1185438, 400106, 136875, 99154, 53463, 37611, 41962]
37611
--- Calculating SSE image = 4, k = 6 with time = 1.6015856266021729 ----
Coverting 5/101
[1154677, 232953, 70128, 40387, 78341, 292756, 143622]
40387
--- Calculating SSE image = 5, k = 4 with time = 1.6026978492736816 ----
Coverting 6/101
[1053785, 429631, 402982, 29452, 31217, 28955, 54578]
28955
--- Calculating SSE image = 6, k = 6 